# Reading & Working with Geometry Files

This notebooks demonstrates how to use the `Grid.from_file()` class method to load in geometry files such as:

1. Shapefile
2. GeoJSON

Highlighted is a workflow showcasing how to remap a variable from an unstructured grid to a Shapefile.

In [ ]:
import uxarray as ux
import warnings
import geocat.datafiles as geodf

warnings.filterwarnings("ignore")

## Load a shapefile and plot  

* This section demonstrates how to load a shapefile using uxarray's Grid.from_file() function
* The shapefile used in this example is the US national boundary file from the US Census Bureau. It is a 20m resolution shapefile that represents the national boundary of the United States. 
* The data plotted is subset to a specific bounding box, which is defined by the latitude and longitude bounds. The result is plotted using the matplotlib backend.

In [ ]:
shp_filename = (
    "../../test/meshfiles/shp/cb_2018_us_nation_20m/cb_2018_us_nation_20m.shp"
)
uxds = ux.Grid.from_file(shp_filename)
lat_bounds = [-90, -70]
lon_bounds = [20, 55]
uxds = uxds.subset.bounding_box(lon_bounds, lat_bounds)
uxds.plot(
    title="US 20m Focus on Mainland (cb_2018_us_nation_20m.shp)",
    backend="matplotlib",
    width=500,
)

## Load a Geojson file and plot

 * This section demonstrates how to load a Geojson file using uxarray's Grid.from_file() function
 * The Geojson file used in this example is a few buildings around downtown Chicago. The plot is shown using the "matplotlib" backend for bounds specific to the region.



In [ ]:
geojson_filename = "../../test/meshfiles/geojson/sample_chicago_buildings.geojson"
uxgeojson = ux.Grid.from_file(geojson_filename)
lat_bounds = [41.6, 42.1]
lon_bounds = [-87.7, -87.5]
uxgeojson.subset.bounding_box(lon_bounds, lat_bounds).plot(backend="matplotlib")

## Open NetCDF mesh file using the Grid.from_file() function

* Regular NetCDF files can also be opened using this function. Backend options available are:
    * xarray
    * geopandas (default for opening shapefile, geojson file and other file formats supported by geopandas read_file function)
* In the following code, we load a NetCDF mesh file: scrip/outCSne8/outCSne8.nc and print out the grid contents.

In [ ]:
nc_filename = "../../test/meshfiles/scrip/outCSne8/outCSne8.nc"
uxgrid = ux.Grid.from_file(nc_filename, backend="xarray")
uxgrid

## Remapping from Shapefile

The following steps are needed for Remapping Global Relative Humidity Data on to a specific region defined by Shapefile using UXarray

1. **Read the shapefile** (uxds)


In [ ]:
shp_filename = (
    "../../test/meshfiles/shp/chicago_neighborhoods/chicago_neighborhoods.shp"
)
uxds = ux.Grid.from_file(shp_filename)
lat_bounds = [41, 43]
lon_bounds = [-89, -90]
uxds = uxds.subset.bounding_box(lon_bounds, lat_bounds)
uxds.plot(
    title="Chicago Neighborhoods",
    backend="bokeh",
)


2. **Initialize Input Data Files**
   - The input datasets consist of NetCDF files that include global relative humidity data.
   - The `datafiles` variable points to two NetCDF files using the `geodf.get` function, specifying the paths:
     - The first file contains meteorological diagnostic data:  
       `netcdf_files/MPAS/FalkoJudt/dyamond_1/30km/diag.2016-08-20_00.00.00_subset.nc`.
     - The second file provides the MPAS grid specification:  
       `netcdf_files/MPAS/FalkoJudt/dyamond_1/30km/x1.655362.grid_subset.nc`.

2. **Open the Datasets with UXarray**
   - The `ux.open_dataset()` function is used to load these files, making them accessible as a UXarray Dataset.
   - `uxds_source` is the opened dataset that holds the meteorological data, such as relative humidity, structured over the MPAS grid.

In [ ]:
datafiles = (
    geodf.get(
        "netcdf_files/MPAS/FalkoJudt/dyamond_1/30km/diag.2016-08-20_00.00.00_subset.nc"
    ),
    geodf.get("netcdf_files/MPAS/FalkoJudt/dyamond_1/30km/x1.655362.grid_subset.nc"),
)

uxds_source = ux.open_dataset(datafiles[1], datafiles[0])

4. **Remap Relative Humidity Data**
   - The `relhum_200hPa` variable is accessed from `uxds_source` to extract relative humidity data at 200 hPa pressure level.
   - **Inverse Distance Weighted Remapping**:
     - The data is remapped using UXarray’s `inverse_distance_weighted` method.
     - The remapping is done to "face centers," adapting the data from its original grid to align with a new shape or structure.

5. **Plot the Remapped Data**
   - The remapped data for Chicago neighborhoods is plotted using UXarray's plotting utilities.
   - The plot uses the `sequential_blue` colormap and is rendered with the `bokeh` backend.
   - The title of the plot is "Chicago Neighborhoods Relative Humidity," giving a clear representation of how relative humidity varies spatially.

In [ ]:
chicago_relative_humidty = uxds_source["relhum_200hPa"].remap.inverse_distance_weighted(
    uxds, remap_to="face centers"
)

chicago_relative_humidty[0].plot(
    cmap=ux.cmaps.sequential_blue,
    title="Chicago Neighborhoods Relative Humidty",
    backend="bokeh",
)